In [1]:
!pip install -q scipy

In [2]:
%%writefile lab_iqr_relatorio.py

from __future__ import annotations
import io
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd

try:
    from scipy.stats import shapiro
    _HAVE_SCIPY = True
except Exception:
    _HAVE_SCIPY = False

@dataclass
class NormalityHeuristics:
    mean_median_ok: bool
    skew_ok: bool
    kurtosis_ok: bool
    shapiro_ok: Optional[bool]
    explanation: str
    @property
    def approx_normal(self) -> bool:
        checks = [self.mean_median_ok, self.skew_ok, self.kurtosis_ok]
        if self.shapiro_ok is not None:
            checks.append(self.shapiro_ok)
        return all(checks)

def _median_absolute_deviation(x: np.ndarray) -> float:
    x = x[~np.isnan(x)]
    if x.size == 0: return np.nan
    med = np.median(x)
    return float(np.median(np.abs(x - med)))

def _quartile_shape(q1: float, q2: float, q3: float) -> str:
    left, right = q2 - q1, q3 - q2
    if left <= 0 or right <= 0: return "indefinida"
    rel_diff = abs(left - right) / max(left, right)
    if rel_diff <= 0.15: return "simétrica"
    if right > left * 1.2: return "cauda à direita (assimetria positiva)"
    if left > right * 1.2: return "cauda à esquerda (assimetria negativa)"
    return "levemente assimétrica"

def _tukey_outliers(x: np.ndarray) -> Tuple[int, float, float]:
    x = x[~np.isnan(x)]
    if x.size == 0: return 0, np.nan, np.nan
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
    n_out = int(((x < low) | (x > high)).sum())
    return n_out, float(low), float(high)

def _bowley_skew(q1: float, q2: float, q3: float) -> float:
    denom = (q3 - q1)
    if denom == 0: return np.nan
    return float((q3 + q1 - 2*q2) / denom)

def _normality_checks(x: np.ndarray, mean: float, median: float, std: float,
                      skew: float, kurtosis_excess: float,
                      use_shapiro: bool) -> NormalityHeuristics:
    parts = []
    mean_median_ok = abs(mean - median) <= 0.25*std if np.isfinite(std) and std > 0 else False
    parts.append(f"- |média−mediana| = {abs(mean-median):.4g} {'≤' if mean_median_ok else '>'} 0,25·DP ({0.25*std:.4g})")
    skew_ok = abs(skew) <= 0.5 if np.isfinite(skew) else False
    parts.append(f"- |skew| = {abs(skew):.4g} {'≤' if skew_ok else '>'} 0,5")
    kurtosis_ok = (-1 <= kurtosis_excess <= 1) if np.isfinite(kurtosis_excess) else False
    parts.append(f"- curtose(excesso) = {kurtosis_excess:.4g} {'∈' if kurtosis_ok else '∉'} [-1, 1]")
    shapiro_ok: Optional[bool] = None
    if use_shapiro and _HAVE_SCIPY and x.size <= 5000 and x.size >= 3:
        try:
            _, p = shapiro(x.astype(float))
            shapiro_ok = bool(p > 0.05)
            parts.append(f"- Shapiro-Wilk p={p:.4g} ⇒ {'não rejeita' if shapiro_ok else 'rejeita'} normalidade a 5%")
        except Exception as e:
            parts.append(f"- Shapiro-Wilk não executado ({e})")
            shapiro_ok = None
    elif use_shapiro and not _HAVE_SCIPY:
        parts.append("- Shapiro-Wilk indisponível (SciPy não instalado)")
    elif use_shapiro and x.size > 5000:
        parts.append("- Shapiro-Wilk não aplicado (n>5000)")
    return NormalityHeuristics(mean_median_ok, skew_ok, kurtosis_ok, shapiro_ok, "\n".join(parts))

def _analyze_numeric(col: pd.Series, use_shapiro: bool) -> Dict[str, object]:
    x = pd.to_numeric(col, errors="coerce").to_numpy(dtype=float)
    n = np.isfinite(x).sum()
    if n == 0: return {"empty": True}
    q1, q2, q3 = np.nanpercentile(x, [25, 50, 75])
    vmin, vmax = np.nanmin(x), np.nanmax(x)
    mean = float(np.nanmean(x))
    std  = float(np.nanstd(x, ddof=1)) if n > 1 else np.nan
    var  = float(np.nanvar(x, ddof=1)) if n > 1 else np.nan
    iqr  = float(q3 - q1)
    mad  = _median_absolute_deviation(x)
    skew = float(pd.Series(x).skew()) if n > 2 else np.nan
    kurt = float(pd.Series(x).kurt()) if n > 3 else np.nan
    bowley = _bowley_skew(q1, q2, q3)
    n_out, low, high = _tukey_outliers(x)
    heur = _normality_checks(x, mean, q2, std if np.isfinite(std) else 0.0,
                             skew if np.isfinite(skew) else np.nan,
                             kurt if np.isfinite(kurt) else np.nan,
                             use_shapiro)
    shape = _quartile_shape(q1, q2, q3)
    return {"empty": False, "count": int(n),
            "min": float(vmin), "q1": float(q1), "median": float(q2), "q3": float(q3), "max": float(vmax),
            "mean": mean, "std": std, "var": var, "iqr": iqr, "mad": mad,
            "skew": skew, "kurtosis_excess": kurt, "bowley_skew": bowley,
            "tukey_outliers": int(n_out), "tukey_low": low, "tukey_high": high,
            "shape": shape, "normality": heur}

def _analyze_categorical(col: pd.Series, top_k: int = 5) -> Dict[str, object]:
    s = col.astype("object")
    total = int(s.shape[0])
    vc = s.value_counts(dropna=False)
    top = vc.head(top_k)
    top_list = [(str(idx), int(cnt), float(cnt/total*100.0)) for idx, cnt in top.items()]
    dominant_prop = float(vc.iloc[0]/total) if total > 0 and len(vc) > 0 else np.nan
    return {"total": total, "top_k": top_list, "dominant_share": dominant_prop}

def _format_numeric_report(name: str, st: Dict[str, object]) -> str:
    norm = st["normality"]
    lines = [
        f"[Numérica] {name} — n={st['count']}",
        f"  Locação: min={st['min']:.6g}, Q1={st['q1']:.6g}, mediana={st['median']:.6g}, Q3={st['q3']:.6g}, máx={st['max']:.6g}",
        f"  Média/Dispersão: média={st['mean']:.6g}, DP={st['std']:.6g}, variância={st['var']:.6g}, IQR={st['iqr']:.6g}, MAD={st['mad']:.6g}",
        f"  Distribuição: skew={st['skew']:.6g}, curtose(excesso)={st['kurtosis_excess']:.6g}, Bowley={st['bowley_skew']:.6g}",
        f"  Outliers (Tukey 1,5×IQR): {st['tukey_outliers']}  [lim_inf={st['tukey_low']:.6g}, lim_sup={st['tukey_high']:.6g}]",
        f"  Forma (quartis): {st['shape']}",
        "  Aproximadamente normal? " + ("Sim" if norm.approx_normal else "Não"),
        "    Critérios:",
        "    " + norm.explanation.replace("\n", "\n    "),
    ]
    concl = []
    if st["tukey_outliers"] > 0: concl.append("há outliers por Tukey")
    if not norm.approx_normal: concl.append("desvia da normalidade")
    if st["iqr"] == 0: concl.append("baixa variabilidade (IQR=0)")
    lines.append("  Conclusão: " + ("; ".join(concl) + "." if concl else "distribuição estável e aproximadamente normal."))
    return "\n".join(lines)

def _format_categorical_report(name: str, info: Dict[str, object]) -> str:
    lines = [f"[Categórica] {name} — n={info['total']}", "  Top-5 categorias:"]
    for val, cnt, pct in info["top_k"]:
        lines.append(f"    - {val}: {cnt} ({pct:.2f}%)")
    dom = info["dominant_share"]
    if np.isfinite(dom):
        lines.append(f"  Diversidade: categoria dominante com {dom*100:.2f}% do total "
                     f"({'alta concentração' if dom >= 0.6 else 'diversidade moderada' if dom >= 0.35 else 'alta diversidade'})")
    return "\n".join(lines)

def generate_report(df: pd.DataFrame, dataset_name: Optional[str] = None,
                    normality_tests: bool = True,
                    include_examples_in_report: bool = False,
                    save_path: Optional[str] = None) -> str:
    if not isinstance(df, pd.DataFrame):
        raise TypeError("df deve ser um pandas.DataFrame")
    n_rows, n_cols = df.shape
    missing_total = int(df.isna().sum().sum())
    missing_pct = float(missing_total / (n_rows * max(n_cols, 1)) * 100.0) if n_rows > 0 and n_cols > 0 else 0.0
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = [c for c in df.columns if c not in numeric_cols]

    buff = io.StringIO()
    title = dataset_name or "dataset"
    buff.write(f"RELATÓRIO EDA — {title}\n")
    buff.write("=" * (15 + len(title)) + "\n\n")
    buff.write("Sumário do dataset\n------------------\n")
    buff.write(f"- Linhas: {n_rows:,}\n- Colunas: {n_cols:,}\n")
    buff.write(f"- Valores faltantes (total): {missing_total:,} ({missing_pct:.2f}%)\n")
    buff.write(f"- Colunas numéricas: {len(numeric_cols)}\n- Colunas categóricas: {len(categorical_cols)}\n\n")

    buff.write("Detalhes por coluna numérica\n----------------------------\n")
    non_normal_cols, tukey_out_cols = [], []
    for col in numeric_cols:
        st = _analyze_numeric(df[col], normality_tests)
        if st.get("empty"):
            buff.write(f"[Numérica] {col}: sem dados válidos\n\n"); continue
        buff.write(_format_numeric_report(col, st) + "\n\n")
        if not st["normality"].approx_normal: non_normal_cols.append(col)
        if st["tukey_outliers"] > 0: tukey_out_cols.append(col)

    buff.write("Detalhes por coluna categórica\n------------------------------\n")
    for col in categorical_cols:
        info = _analyze_categorical(df[col], top_k=5)
        buff.write(_format_categorical_report(col, info) + "\n\n")

    buff.write("Conclusões gerais\n-----------------\n")
    if missing_total > 0:
        buff.write(f"- Há {missing_total:,} valores faltantes ({missing_pct:.2f}%). Considere imputação ou remoção.\n")
    else:
        buff.write("- Não há valores faltantes.\n")
    if non_normal_cols:
        buff.write(f"- Colunas que **não** parecem aproximadamente normais: {', '.join(non_normal_cols)}.\n")
    else:
        buff.write("- Todas as colunas numéricas parecem aproximadamente normais pelos critérios.\n")
    if tukey_out_cols:
        buff.write(f"- Colunas com outliers (Tukey 1,5×IQR): {', '.join(tukey_out_cols)}.\n")
    else:
        buff.write("- Não foram detectados outliers pela regra de Tukey.\n")
    buff.write("- Próximos passos sugeridos:\n")
    buff.write("  * Verificar impacto de outliers; winsorizar/transformar se necessário.\n")
    buff.write("  * Em não normais, considerar log/Box-Cox/Yeo-Johnson ou métodos não paramétricos.\n")
    buff.write("  * Em categóricas concentradas, reagrupamento de raras.\n\n")
    buff.write("Observações didáticas\n---------------------\n")
    buff.write("- IQR (Q3−Q1) cobre os 50% centrais; Tukey marca outliers fora de [Q1−1,5·IQR, Q3+1,5·IQR].\n")
    buff.write("- Quartis & simetria: Q2 central ⇒ simétrica; deslocamentos ⇒ caudas.\n")
    buff.write("- Skew (3º momento) → assimetria; curtose (4º) → caudas/achatamento.\n")
    buff.write("- 'Aproximadamente normal' segue os critérios e pode usar Shapiro quando aplicável.\n")
    buff.write("- Categóricas: concentração alta pode prejudicar modelos; diversidade ajuda.\n")

    text = buff.getvalue()
    if save_path:
        with open(save_path, "w", encoding="utf-8") as f: f.write(text)
    return text

__all__ = ["generate_report"]

Writing lab_iqr_relatorio.py


In [3]:
import numpy as np, pandas as pd
rng = np.random.default_rng(7)
n = 1500
df_csv = pd.DataFrame({
    "id": np.arange(1, n+1),
    "idade": rng.normal(34, 11, n).round().astype(int),
    "salario": rng.lognormal(10.1, 0.55, n),
    "departamento": rng.choice(["Vendas","Suporte","Engenharia","Marketing"], size=n, p=[0.35,0.25,0.30,0.10]),
    "cidade": rng.choice(["Fortaleza","Sobral","Juazeiro","Crato","Iguatu"], size=n, p=[0.5,0.15,0.15,0.1,0.1]),
    "score": rng.beta(2,5,n)*100,
    "satisfacao_1a5": rng.choice([1,2,3,4,5], size=n, p=[0.1,0.15,0.3,0.3,0.15]),
    "gasto_mensal": rng.gamma(2.2,1200,n),
    "compras": rng.poisson(3.2,n),
    "churn": rng.choice(["Sim","Não"], size=n, p=[0.22,0.78]),
    "data_cadastro": pd.date_range("2021-01-01", periods=n, freq="D"),
})
df_csv.loc[::40,"salario"] = np.nan
df_csv.loc[::55,"departamento"] = None
df_csv.loc[::70,"satisfacao_1a5"] = np.nan
df_csv.loc[::33,"gasto_mensal"] = np.nan
df_csv.to_csv("dados_exemplo_eda.csv", index=False)
df_csv.head()

,id,idade,salario,departamento,cidade,score,satisfacao_1a5,gasto_mensal,compras,churn,data_cadastro
0,1,34,NaN,None,Fortaleza,12.726712,NaN,NaN,2,Não,2021-01-01
1,2,37,60145.180654,Vendas,Crato,21.295491,5.0,5186.418963,3,Não,2021-01-02
2,3,31,32171.702000,Suporte,Fortaleza,7.100370,1.0,985.595148,5,Não,2021-01-03
3,4,24,65113.048316,Vendas,Iguatu,22.481196,4.0,1459.829301,4,Não,2021-01-04
4,5,29,37732.881651,Vendas,Juazeiro,43.688414,5.0,2119.300424,5,Não,2021-01-05


In [4]:
from importlib import reload
import lab_iqr_relatorio as lir
reload(lir)

texto = lir.generate_report(
    df_csv,
    dataset_name="dados_exemplo_eda",
    normality_tests=True,
    save_path="relatorio_eda_dados_exemplo.txt"
)
print(texto[:1500])

RELATÓRIO EDA — dados_exemplo_eda

Sumário do dataset
------------------
- Linhas: 1,500
- Colunas: 11
- Valores faltantes (total): 134 (0.81%)
- Colunas numéricas: 7
- Colunas categóricas: 4

Detalhes por coluna numérica
----------------------------
[Numérica] id — n=1500
  Locação: min=1, Q1=375.75, mediana=750.5, Q3=1125.25, máx=1500
  Média/Dispersão: média=750.5, DP=433.157, variância=187625, IQR=749.5, MAD=375
  Distribuição: skew=0, curtose(excesso)=-1.2, Bowley=0
  Outliers (Tukey 1,5×IQR): 0  [lim_inf=-748.5, lim_sup=2249.5]
  Forma (quartis): simétrica
  Aproximadamente normal? Não
    Critérios:
    - |média−mediana| = 0 ≤ 0,25·DP (108.3)
    - |skew| = 0 ≤ 0,5
    - curtose(excesso) = -1.2 ∉ [-1, 1]
    - Shapiro-Wilk p=4.062e-21 ⇒ rejeita normalidade a 5%
  Conclusão: desvia da normalidade.

[Numérica] idade — n=1500
  Locação: min=-2, Q1=26, mediana=33, Q3=41, máx=65
  Média/Dispersão: média=33.3853, DP=10.7938, variância=116.505, IQR=15, MAD=7
  Distribuição: skew=0.0768

/content/lab_iqr_relatorio.py:113: FutureWarning: The behavior of value_counts with object-dtype is deprecated. In a future version, this will *not* perform dtype inference on the resulting index. To retain the old behavior, use `result.index = result.index.infer_objects()`
  vc = s.value_counts(dropna=False)
